In [ ]:
import os
import shutil
from PIL import Image
import numpy as np
import tensorflow as tf
import keras
from keras_preprocessing import image
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from cv2 import cv2
import matplotlib.pyplot as plt
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
#load trained model
model = keras.models.load_model('trianedmodel.h5')

In [ ]:
#generate augmented image dataset
#Only one run is required for a new data set.
datagen=ImageDataGenerator(
    rotation_range = 45,     # Random rotation degree
    width_shift_range = 0.2, # Random horizontal translation
    height_shift_range = 0.2,# Random vertical translation
    rescale = 1/255,         # Data Normalization
    shear_range = 20,       # Random tangent transformation
    zoom_range = 0.4,        # Random zoom in
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest',
)

datapath = 'fruit-and-vegetable-image-recognition/train'
    
for lable in os.listdir(datapath):
    lablepath = datapath+'/'+lable
 
    if os.path.exists(lablepath+'/augmented') is False:      
        os.makedirs(lablepath+'/augmented')
        
    for file in os.listdir(lablepath):
        filepath = lablepath+'/'+file
        if os.path.isfile(filepath):
            img = load_img(filepath)
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)
            i = 0
            for batch in datagen.flow(x, batch_size=1,
                                      save_to_dir=lablepath+'/augmented',
                                      save_prefix='augmented',
                                      save_format='jpg',
                                      ignore_class_split = True):
                i += 1
                if i > 10:
                    break

for lable in os.listdir(datapath):
    orifilepath = datapath+'/'+lable+'/augmented'
    desfilepath = datapath+'/'+lable
    get_files = os.listdir(orifilepath)
    for file in get_files:
        shutil.move(orifilepath + '/'+file, desfilepath)

In [ ]:
#Generates a tf.data.Dataset from augmented image files in a directory
val_path="./fruit-and-vegetable-image-recognition/validation"
train_path="./fruit-and-vegetable-image-recognition/train"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(train_path,
                                                               seed=317,
                                                               image_size=(224, 224),
                                                              batch_size=32)
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(val_path,
                                                              seed=317,
                                                              image_size=(224, 224),
                                                              shuffle=False,
                                                              batch_size=32)
#show labels
class_names = train_dataset.class_names
print("Lables: " + str(class_names))

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size = (5, 5), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(class_names),activation='sigmoid'))

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adamax(lr=0.001,decay=0.0),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),   
    metrics = ["accuracy"])

In [ ]:
history = model.fit(x=train_dataset,
                    epochs= 30,
                    validation_data=val_dataset)

In [ ]:
model.save('trianedmodel.h5')

In [ ]:
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel('Time')
plt.legend(['val_loss', 'loss'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['accuracy'])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel('Time')
plt.legend(['val_accuracy', 'acc'], loc='upper left')
plt.show()

In [ ]:
def test(image_path, model):
    img = image.load_img(image_path, target_size=(224,224,3))
    plt.imshow(img)
    plt.show()
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    
    pred = model.predict(images)
    predict = model.predict(x)
    
    top5 = np.argsort(pred, axis=1)[:,-5:]
    top5label = []
    i = 4
    while i > -1:
        top5label.append(class_names[(top5[0][i])])
        i = i-1
    
    actuallabel0 = image_path.split("/")[-1]
    actuallabel = actuallabel0.split(".")[-2]
    if actuallabel == class_names[np.argmax(pred)]:
        top1result = "The Top1 prediction was successful\n"
    else:
        top1result = "The Top1 prediction was unsuccessful\n"
        
    for i in top5label:
        if i == actuallabel:
            top5result = "The Top5 prediction was successful\n"
            break
        else:
            top5result = "The Top5 prediction was unsuccessful\n"
            
    print("\nActual: "+ actuallabel +"\n")
    print("Top 1 predict: "+class_names[np.argmax(pred)])
    print(top1result)
    print("Top 5 predict: "+str(top5label))
    print(top5result)
    return

In [ ]:
test("./test/corn.jpg", model)

In [ ]:
test("./test/banana.jpg", model)

In [ ]:
test("./test/apple.jpg", model)

In [ ]:
test("./test/tomato.jpg", model)